In [10]:
# https://towardsdatascience.com/python-how-to-get-live-market-data-less-than-0-1-second-lag-c85ee280ed93

In [1]:
import numpy as np
import pandas as pd

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go

https://aroussi.com/post/python-yahoo-finance

#### 通过ticker对接数据

In [2]:
cingf = yf.Ticker("COIN.A")
print(cingf)

yfinance.Ticker object <COIN.A>


In [3]:
cingf.info

HTTPError: HTTP Error 404: Not Found

#### 要将历史数据另存成pandas df，方便今后操作
Available paramaters for the history() method are:

period: data period to download (Either Use period parameter or use start and end) Valid periods are: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max

interval: data interval (intraday data cannot extend last 60 days) Valid intervals are: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo

start: If not using period - Download start date string (YYYY-MM-DD) or datetime.

end: If not using period - Download end date string (YYYY-MM-DD) or datetime.

prepost: Include Pre and Post market data in results? (Default is False)

auto_adjust: Adjust all OHLC automatically? (Default is True)

actions: Download stock dividends and stock splits events? (Default is True)


In [4]:
msft_hist = msft.history(period='max')

NameError: name 'msft' is not defined

#### 直接获取分红和配股信息

In [224]:
msft.actions.head()
# msft.dividends
# msft.splits

Date
1987-09-21    2.0
1990-04-16    2.0
1991-06-27    1.5
1992-06-15    1.5
1994-05-23    2.0
1996-12-09    2.0
1998-02-23    2.0
1999-03-29    2.0
2003-02-18    2.0
Name: Stock Splits, dtype: float64

#### 通过pd df 更好的切片，分析数据

In [220]:
msft_hist[(msft_hist['Dividends'] >0) | (msft_hist['Stock Splits'] >0)][['Dividends','Stock Splits']].describe()

,Dividends,Stock Splits
count,78.000000,78.000000
mean,0.259872,0.217949
std,0.361259,0.611522
min,0.000000,0.000000
25%,0.100000,0.000000
50%,0.180000,0.000000
75%,0.360000,0.000000
max,3.080000,2.000000


#### index是datetime形式的，可以按如下筛选切片

In [213]:
msft_hist[('2021-03-12' > msft_hist.index)& (msft_hist.index > '2021-03-10')]

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-03-11,234.960007,239.169998,234.309998,237.130005,29896000,0.0,0.0


In [225]:
# data = yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30", group_by="ticker") #改变multi-columns的顺序，让AAPL和SPY变成一级引索

data = yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30")

[*********************100%***********************]  2 of 2 completed


In [227]:
data.head()

Adj Close                  Close                   High  \
                 AAPL         SPY       AAPL         SPY       AAPL   
Date                                                                  
2017-01-03  27.459938  208.213501  29.037500  225.240005  29.082500   
2017-01-04  27.429203  209.452240  29.004999  226.580002  29.127501   
2017-01-05  27.568691  209.285812  29.152500  226.399994  29.215000   
2017-01-06  27.876030  210.034561  29.477501  227.210007  29.540001   
2017-01-09  28.131361  209.341278  29.747499  226.460007  29.857500   

                              Low                   Open              \
                   SPY       AAPL         SPY       AAPL         SPY   
Date                                                                   
2017-01-03  225.830002  28.690001  223.880005  28.950001  225.039993   
2017-01-04  226.750000  28.937500  225.610001  28.962500  225.619995   
2017-01-05  226.580002  28.952499  225.479996  28.980000  226.270004   
2017-01-06  227.750000  29.117500  225.899994  29.195000  226.529999   
2017-01-09  227.070007  29.485001  226.419998  29.487499  226.910004   

               Volume            
                 AAPL       SPY  
Date                             
2017-01-03  115127600  91366500  
2017-01-04   84472400  78744400  
2017-01-05   88774400  78379000  
2017-01-06  127007600  71559900  
2017-01-09  134247600  46939700

In [230]:
data['Close']['SPY']

Date
2017-01-03    225.240005
2017-01-04    226.580002
2017-01-05    226.399994
2017-01-06    227.210007
2017-01-09    226.460007
                 ...    
2017-04-24    237.169998
2017-04-25    238.550003
2017-04-26    238.399994
2017-04-27    238.600006
2017-04-28    238.080002
Name: SPY, Length: 81, dtype: float64

In [244]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html
idx = pd.IndexSlice
data.loc[:, idx[:,'SPY']]

,Adj Close,Close,High,Low,Open,Volume
,SPY,SPY,SPY,SPY,SPY,SPY
Date,,,,,,
2017-01-03,208.213501,225.240005,225.830002,223.880005,225.039993,91366500
2017-01-04,209.452240,226.580002,226.750000,225.610001,225.619995,78744400
2017-01-05,209.285812,226.399994,226.580002,225.479996,226.270004,78379000
2017-01-06,210.034561,227.210007,227.750000,225.899994,226.529999,71559900
2017-01-09,209.341278,226.460007,227.070007,226.419998,226.910004,46939700
...,...,...,...,...,...,...
2017-04-24,220.195480,237.169998,237.410004,234.559998,237.179993,119209900
2017-04-25,221.476685,238.550003,238.949997,237.809998,237.910004,76698300


In [245]:
SPY = data.loc[:, idx[:,'SPY']]
SPY.columns

MultiIndex([('Adj Close', 'SPY'),
            (    'Close', 'SPY'),
            (     'High', 'SPY'),
            (      'Low', 'SPY'),
            (     'Open', 'SPY'),
            (   'Volume', 'SPY')],
           )

In [246]:
col_names = ['adj Close','Close', 'High', 'Low','Open','Volume']
SPY.columns = col_names

In [247]:
SPY.head()

,adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2017-01-03,208.213501,225.240005,225.830002,223.880005,225.039993,91366500
2017-01-04,209.452240,226.580002,226.750000,225.610001,225.619995,78744400
2017-01-05,209.285812,226.399994,226.580002,225.479996,226.270004,78379000
2017-01-06,210.034561,227.210007,227.750000,225.899994,226.529999,71559900
2017-01-09,209.341278,226.460007,227.070007,226.419998,226.910004,46939700


#### 下载后直接存成csv文件
可以先存成变量，再存成csv

或者可以一步到位，直接存成csv

In [249]:
spy = yf.download("SPY", start="2017-01-01", end="2017-04-30")

[*********************100%***********************]  1 of 1 completed


In [251]:
# spy.to_csv(r'/Users/paxton615/Github_Personal/Notes/Notes/Financial_models/spy_070101_070403.csv')

In [252]:
# yf.download("aapl", period='max').to_csv(r'/Users/paxton615/Github_Personal/Notes/Notes/Financial_models/apple_max.csv')

[*********************100%***********************]  1 of 1 completed


### 如果想从yf切换回pandas_datareader

In [ ]:

# from pandas_datareader import data as pdr

# import yfinance as yf
# yf.pdr_override() # <== that's all it takes :-)

# # download dataframe using pandas_datareader
# data = pdr.get_data_yahoo("SPY", start="2017-01-01", end="2017-04-30")

#### 升级yfinance
！ pip install yfinance --upgrade --no-cache-dir